# Coordinate da Indirizzo

In [2]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Novegro 20090 Segrate")
start_lat=location.latitude 
start_lng=location.longitude
print(start_lat,start_lng)


45.4688535 9.2776371


In [12]:
import geopy.geocoders
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Como 9 Barzago")
end_lat=location.latitude 
end_lng=location.longitude
print(end_lat,end_lng)

45.7578372 9.3097403


# Calcolo percorso più veloce

In [14]:
#Find Distance
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
%matplotlib inline

np.random.seed(0)
ox.__version__

ox.settings.log_console=True
ox.settings.cache_folder='C:\\Users\\lucaf\\OneDrive\\Desktop\\Esercizi\\Cache'
place = "Milan,Lombardy,Italy"
mode="drive"
G = ox.graph_from_place(place, network_type=mode)
Gp = ox.project_graph(G)

KeyboardInterrupt: 

In [85]:
orig_point = (start_lat, start_lng) # UTM coordinates
dest_point = (end_lat, end_lng)

orig_node, dest_node = ox.nearest_nodes(
                            Gp,
                            X=[orig_point[0], dest_point[0]],
                            Y=[orig_point[1], dest_point[1]]
                                       )

In [86]:
orig_edge, dest_edge = ox.nearest_edges(
                            Gp,
                            X=[orig_point[0], dest_point[0]],
                            Y=[orig_point[1], dest_point[1]]
                                       )

In [87]:
print(orig_point,dest_point)
print(orig_node,dest_node)

(45.4688535, 9.2776371) (45.7578372, 9.3097403)
583043033 583043033


In [80]:
route = ox.shortest_path(Gp, orig_node, dest_node, weight="length")

In [13]:
shortest_route_map = ox.plot_route_folium(G, route,tiles='openstreetmap')
shortest_route_map

NameError: name 'ox' is not defined

In [ ]:

# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

# same thing again, but this time pass in a few default speed values (km/hour)
# to fill in edges with missing `maxspeed` from OSM
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

# calculate two routes by minimizing travel distance vs travel time
orig = list(G)[1]
dest = list(G)[120]
route1 = ox.shortest_path(G, orig, dest, weight="length")
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

# compare the two routes
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")


In [14]:
import pandas as pd
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
import pandas as pd
import osmnx as ox

ox.settings.cache_folder='C:\\Users\\lucaf\\OneDrive\\Desktop\\Esercizi\\Cache'
places=["Milan,Lombardy,Italy","Segrate,Lombardy,Italy"]
tags = {"public_transport":"platform",
        "bus":"yes",
        "node":True}

all_pois = ox.geometries_from_place(places,tags)
point=pd.DataFrame(all_pois)
point=point[["name","public_transport","geometry"]].reset_index()
point=point[point.element_type=="node"]
point["lat"]=point.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
point["lon"]=point.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])
bus=point.drop_duplicates(subset=['lat','lon'])
point.to_csv("bus_location.csv")

In [15]:
tags2 = {"public_transport":"station"}

all_pois2 = ox.geometries_from_place(places,tags2)
point2=pd.DataFrame(all_pois2).reset_index()
point2=point2[point2.element_type=="node"]
point2=point2[["name","public_transport","geometry","subway","railway"]].reset_index()

point2["lat"]=point2.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
point2["lon"]=point2.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])

train=point2[point2.subway!="yes"]
metro=point2[point2.subway=="yes"]
train.to_csv("train_location.csv")
metro.to_csv("metro_location.csv")

In [16]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import pydeck as pdk
INITIAL_VIEW_STATE = pdk.ViewState(
  latitude=45.47185532715593, 
  longitude=9.275071955673953,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)


In [17]:
origin=pd.DataFrame({"lon":9.275071955673953,"lat":45.47185532715593}, index=[0])
Origin_layer = pdk.Layer(
    'ScatterplotLayer',
    origin,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    get_radius=100,
    get_color='[39, 71, 245,140]',
    pickable=True)

In [18]:
train2=train[["lat","lon"]]
train2.to_csv("train_latlon.csv")
train2["lat"]=train2["lat"].apply(lambda x : float(x))
train2["lon"]=train2["lon"].apply(lambda x : float(x))
Train_Layer = pdk.Layer(
    'ScatterplotLayer',
    train2,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    get_radius=50,
    get_color='[20, 232, 62, 140]',
    pickable=True)

C:\Users\lucaf\AppData\Local\Temp\ipykernel_20784\1477248746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train2["lat"]=train2["lat"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_20784\1477248746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train2["lon"]=train2["lon"].apply(lambda x : float(x))


In [9]:
metro2=metro[["lat","lon"]]
metro2.to_csv("metro_latlon.csv")
metro2["lat"]=metro2["lat"].apply(lambda x : float(x))
metro2["lon"]=metro2["lon"].apply(lambda x : float(x))
Metro_Layer = pdk.Layer(
    'ScatterplotLayer',
    metro2,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    get_radius=50,
    get_color='[225, 232, 21, 140]',
    pickable=True)

C:\Users\lucaf\AppData\Local\Temp\ipykernel_20784\2601925485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro2["lat"]=metro2["lat"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_20784\2601925485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro2["lon"]=metro2["lon"].apply(lambda x : float(x))


In [11]:
bus2=bus[["lat","lon"]]
bus2.to_csv("bus_latlon.csv")
bus2["lat"]=bus2["lat"].apply(lambda x : float(x))
bus2["lon"]=bus2["lon"].apply(lambda x : float(x))
Bus_Layer = pdk.Layer(
    'ScatterplotLayer',
    bus2,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    get_radius=50,
    get_color='[180, 0, 200, 140]',
    pickable=True)

C:\Users\lucaf\AppData\Local\Temp\ipykernel_20784\2707445566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2["lat"]=bus2["lat"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_20784\2707445566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2["lon"]=bus2["lon"].apply(lambda x : float(x))


In [12]:
pdk.Deck(
    layers=[Origin_layer,Bus_Layer,Metro_Layer,Train_Layer],
    initial_view_state=INITIAL_VIEW_STATE)

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 45.47185532715593,
    "longitude": 9.275071955673953,
    "maxZoom": 16,
    "pitch": 45,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "autoHighlight": true,
      "data": [
        {
          "lat": 45.47185532715593,
          "lon": 9.275071955673953
        }
      ],
      "getColor": "@@=[39, 71, 245,140]",
      "getPosition": "@@=[lon, lat]",
      "getRadius": 100,
      "id": "e2eb5a27-66e1-422a-83cf-14d38463f83e",
      "pickable": true
    },
    {
      "@@type": "ScatterplotLayer",
      "autoHighlight": true,
      "data": [
        {
          "lat": 45.4522821,
          "lon": 9.1417609
        },
        {
          "lat": 45.4509566,
          "lon": 9.1687198
        },
        {
          "lat": 45.4787442,
          "lon": 9.1749509
        },
        {
          "lat": 45.4909719,
          "lon": 9.1537332
        },
        {
          "lat": 45.4915303,
          "lon": 9.1534811
        },
        {
          "lat": 45.4934557,
          "lon": 9.1570134
        },
        {
          "lat": 45.4855757,
          "lon": 9.1582607
        },
        {
          "lat": 45.4735476,
          "lon": 9.1561864
        },
        {
          "lat": 45.452174,
          "lon": 9.2242657
        },
        {
          "lat": 45.4446254,
          "lon": 9.1727674
        },
        {
          "lat": 45.4532051,
          "lon": 9.1462362
        },
        {
          "lat": 45.4538109,
          "lon": 9.2239156
        },
        {
          "lat": 45.4595539,
          "lon": 9.223923
        },
        {
          "lat": 45.4658972,
          "lon": 9.223937
        },
        {
          "lat": 45.469675,
          "lon": 9.2238108
        },
        {
          "lat": 45.4734751,
          "lon": 9.2250046
        },
        {
          "lat": 45.4670591,
          "lon": 9.1592314
        },
        {
          "lat": 45.4751033,
          "lon": 9.1365193
        },
        {
          "lat": 45.4695368,
          "lon": 9.1319028
        },
        {
          "lat": 45.4586162,
          "lon": 9.1383612
        },
        {
          "lat": 45.4556353,
          "lon": 9.1400656
        },
        {
          "lat": 45.467439,
          "lon": 9.1655722
        },
        {
          "lat": 45.4622095,
          "lon": 9.1285337
        },
        {
          "lat": 45.4872282,
          "lon": 9.1308342
        },
        {
          "lat": 45.4905915,
          "lon": 9.237132
        },
        {
          "lat": 45.4471073,
          "lon": 9.2033952
        },
        {
          "lat": 45.4946969,
          "lon": 9.2360076
        },
        {
          "lat": 45.4871584,
          "lon": 9.2372883
        },
        {
          "lat": 45.491506,
          "lon": 9.2496702
        },
        {
          "lat": 45.4919517,
          "lon": 9.2368814
        },
        {
          "lat": 45.4995833,
          "lon": 9.2422126
        },
        {
          "lat": 45.4300653,
          "lon": 9.174702
        },
        {
          "lat": 45.4385327,
          "lon": 9.1388557
        },
        {
          "lat": 45.4384349,
          "lon": 9.1406975
        },
        {
          "lat": 45.4319019,
          "lon": 9.1610501
        },
        {
          "lat": 45.4302035,
          "lon": 9.1601674
        },
        {
          "lat": 45.4294992,
          "lon": 9.1570919
        },
        {
          "lat": 45.4766536,
          "lon": 9.086211
        },
        {
          "lat": 45.4660283,
          "lon": 9.1278899
        },
        {
          "lat": 45.4682046,
          "lon": 9.1294994
        },
        {
          "lat": 45.4606086,
          "lon": 9.087483
        },
        {
          "lat": 45.4598402,
          "lon": 9.1061113
        },
        {
          "lat": 45.4600225,
          "lon": 9.129239
        },
        {
          "lat": 45.4643259,
          "lon":